In [1]:
import psycopg2
import pandas as pd
import sys
import re
import numpy as np
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from sklearn.feature_extraction.text import CountVectorizer
import operator
import pymorphy2
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import io
import tokenize as T
from operator import itemgetter
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import gensim
from gensim import corpora
from pprint import pprint

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DDubai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DDubai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DDubai\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
import difflib

In [3]:
from flashtext.keyword import KeywordProcessor

# подключение к постгрес

In [4]:
# Connection parameters, yours will be different
param_dic = {
    "host"      : "***",
    "database"  : "***",
    "user"      : "***",
    "password"  : "***"
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [5]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [13]:
%%time
con = connect(param_dic)
select = '''
SELECT source_system, audio_id, start_second, end_second, transcribation, user_name
from public.calltranscribation
where audio_id in (
SELECT distinct audio_id
FROM public.calltranscribation
where audio_id IN {}
 )
'''

column_name = ['source_system', 'audio_id', 'start_second', 'end_second', 'transcribation', 'user_name']

df = postgresql_to_dataframe(conn = con, select_query = select.format(tuple(id_calls.audio_id.tolist())), column_names = column_name)

Connecting to the PostgreSQL database...
Connection successful
CPU times: total: 328 ms
Wall time: 3min 13s


# Алгоритмы

In [7]:
#выделение контрольных слов
def control_words(corpus):
    vectorizer = CountVectorizer(min_df =1)
    morph = pymorphy2.MorphAnalyzer()
    corpus = corpus.dropna()
    #приводим к нормальной форме
    re_lemm_phrase = re.sub(r'[^\w\s]','', str(corpus.tolist()))
    lemm_phrases = [' '.join([morph.parse(word)[0].normal_form for word in re_lemm_phrase.split()])]
    pure_lemm_phrase = [i for i in lemm_phrases[0].split() if i not in stopwords.words('russian')]
    #составляем Bag of Words
    corpus_tokens = vectorizer.fit_transform(pure_lemm_phrase)
    #смотрим сформированные токены

    matrix_freq = corpus_tokens.toarray().sum(axis=0)
    #итоговая матрица слов и нормированных частот 
    final_matrix = np.array([np.array(vectorizer.get_feature_names_out()),
                            matrix_freq/corpus_tokens.shape[1]], 
                            dtype = object
                        )
    #словарь слов и частот
    bow_dict = dict(zip(final_matrix[0], final_matrix[1]))

    newDict = dict()
    #берем самые популярные слова
    for key, value in bow_dict.items():
        if value>=0.5:
            newDict[key] = value
    control_words = list(newDict.keys())

    return control_words

In [8]:
#наличие вхождения слов в строку
def word_pres_check(keyword_df, main_df):
    
    morph = pymorphy2.MorphAnalyzer()
    #добавление ключевых слов
    keyword_processor = KeywordProcessor()
    keyword_processor.add_keywords_from_list(keyword_df)
    
    #приведение к нормальному виду транскрибации (+удаление знаков препинания и приведение к нижнему регистру)
    lemm_phrase = [' '.join([morph.parse(i)[0].normal_form for i in main_df.split()])]
    #re_lemm_phrase = re.sub(r'[^\w\s]','', lemm_phrase[0])
    
    #удаление стоп слов из лемм нормализованных транскрибаций
    #pure_lemm_phrase = [i for i in lemm_phrase.split() if i not in stopwords.words('russian')]
    
    #поиск вхождений
    set_found_words = set(keyword_processor.extract_keywords(lemm_phrase[0]))
    
    #пересечение и объединение 

    return len(set_found_words), len(keyword_df), set_found_words

# Датафреймы

In [12]:
#айдишники звонков
id_calls = pd.read_excel(r'calls_examples.xlsx')

In [36]:
work_df = (df
           .query("user_name !='' ")
           .groupby(['user_name','audio_id'], as_index = False)['transcribation']
           .apply(lambda x: ' '.join(x))
           .reset_index()
          )

In [37]:
work_df.drop(['index'],axis = 1, inplace = True)

In [39]:
#чек лист диспетчера
disp_check_lst = pd.read_excel(r'disp_check_lst.xlsb')

In [34]:
print('всего уникальных пользователей :', len(df.user_name.unique()),
    '\nвсего уникальных звонков :', len(df.audio_id.unique()),
      '\nзаселекчено уникальных пользователей :',len(ndf.user_name.unique()),
      '\nзаселекчено уникальных звонков :', len(ndf.audio_id.unique())   
     )

всего уникальных пользователей : 8 
всего уникальных звонков : 6096 
заселекчено уникальных пользователей : 7 
заселекчено уникальных звонков : 5961


In [450]:
#рабочий датасет со слитыми транскрибациями
work_df = (
            df
            .query("user_name != ''")
            .groupby(['audio_id','user_name'])['transcribation']
            .apply(lambda x: ' '.join(x))
            .reset_index()
          )

# ключевые слова

In [40]:
%%time
#hello_check = control_words(disp_check_lst['Приветствие'][:4])
performance_check = control_words(disp_check_lst['Представление '])
comfort_check = control_words(disp_check_lst['Удобно ли разговаривать'])
city_check = control_words(disp_check_lst['Уточняющий вопрос(город)'])
lvl_check = control_words(disp_check_lst['Уточняющий вопрос(уровень образования)'])
faculty_check = control_words(disp_check_lst['Уточняющий вопрос(факультет)']) 
trans_check = control_words(disp_check_lst['Перевод']) 

CPU times: total: 969 ms
Wall time: 1.19 s


# проверка чек листа

In [85]:
#удобная функция)))))
def check_foo(df,check_list):
    return(df
           .transcribation
           .apply(lambda x: (word_pres_check(check_list,x)))
          )

In [86]:
%%time
work_df['performance_check'] = check_foo(work_df, performance_check)
work_df['comfort_check'] = check_foo(work_df, comfort_check)
work_df['city_check'] = check_foo(work_df, city_check)
work_df['lvl_check'] = check_foo(work_df, lvl_check)
work_df['faculty_check'] = check_foo(work_df, faculty_check)
work_df['trans_check'] = check_foo(work_df, trans_check)

CPU times: total: 1h 38min 4s
Wall time: 1h 51min 28s


In [99]:
work_df

,user_name,audio_id,transcribation,performance_check,comfort_check,city_check,lvl_check,faculty_check,trans_check
0,aantipova,38296050,грациано университет смерти и меня зовут павел...,"(5, 9, {звать, сайт, оставить, заявка, универс...","(5, 8, {получить, удобно, специалист, подсказа...","(2, 5, {подсказать, пожалуйста})","(2, 7, {подсказать, пожалуйста})","(2, 8, {подсказать, пожалуйста})","(0, 2, {})"
1,aantipova,38296057,броски екатерина университет энергии меня зову...,"(5, 9, {звать, сайт, оставить, заявка, универс...","(5, 8, {получить, удобно, специалист, подсказа...","(2, 5, {подсказать, пожалуйста})","(2, 7, {подсказать, пожалуйста})","(2, 8, {подсказать, пожалуйста})","(0, 2, {})"
2,aantipova,38296060,здравствуй ольга университет синергия меня зов...,"(6, 9, {звать, синергия, сайт, оставить, заявк...","(5, 8, {получить, удобно, специалист, подсказа...","(4, 5, {подсказать, город, пожалуйста, прожива...","(3, 7, {подсказать, пожалуйста, интересовать})","(3, 8, {подсказать, пожалуйста, интересовать})","(1, 2, {соединять})"
3,aantipova,38296064,здравствуйте университете энергии меня зовут п...,"(5, 9, {звать, сайт, оставить, заявка, универс...","(5, 8, {получить, удобно, специалист, подсказа...","(2, 5, {подсказать, пожалуйста})","(2, 7, {подсказать, пожалуйста})","(2, 8, {подсказать, пожалуйста})","(0, 2, {})"
4,aantipova,38296069,здравствуй евгений университет энергии меня зо...,"(5, 9, {звать, сайт, оставить, заявка, универс...","(5, 8, {получить, удобно, специалист, подсказа...","(4, 5, {подсказать, город, пожалуйста, прожива...","(4, 7, {подсказать, рассматривать, пожалуйста,...","(5, 8, {рассматривать, направление, интересова...","(0, 2, {})"
...,...,...,...,...,...,...,...,...,...
5959,varutiunian,39845063,татьяна здравствуйте от форума иду нетворк мен...,"(6, 9, {звать, синергия, сайт, оставить, заявк...","(2, 8, {подсказать, специалист})","(2, 5, {подсказать, город})","(2, 7, {подсказать, интересовать})","(3, 8, {подсказать, факультет, интересовать})","(0, 2, {})"
5960,varutiunian,39864976,мария здравствуйте просто платформа игнат варг...,"(5, 9, {звать, сайт, оставить, заявка, универс...","(5, 8, {получить, удобно, специалист, подсказа...","(2, 5, {подсказать, сказать})","(4, 7, {подсказать, рассматривать, сказать, ин...","(6, 8, {рассматривать, факультет, направление,...","(0, 2, {})"
5961,varutiunian,39872426,яна здравствуйте платформа еду нетворк меня зо...,"(7, 9, {звать, заявка, синергия, сайт, оставит...","(6, 8, {получить, удобно, пожалуйста, специали...","(5, 5, {город, сказать, проживать, подсказать,...","(6, 7, {рассматривать, пожалуйста, образование...","(6, 8, {рассматривать, факультет, направление,...","(1, 2, {линия})"
5962,varutiunian,39896647,добер спасибо большое за ожидание уточнила инф...,"(2, 9, {оставить, заявка})","(2, 8, {сказать, специалист})","(2, 5, {город, сказать})","(2, 7, {рассматривать, сказать})","(3, 8, {факультет, рассматривать, сказать})","(0, 2, {})"
